In [5]:
import requests
import json
import sqlite3
import os
import json
import base64

In [6]:
#define image path
image_path = "D:\AIML\Tutorial\galgadot.jpg"

# Open the image file in binary mode, encode it in base64, and decode it to ASCII
with open(image_path, "rb") as image_file:
    encoded_string = base64.b64encode(image_file.read()).decode('ascii')

In [7]:
# Call the API and get the generated text
api_endpoint = 'http://148.113.143.16:8080'

# Prepare the headers for the API request
headers = {
	'Content-Type': 'application/json',
}

# Prepare the payload for the API request
payload = {
	"image": encoded_string
}

In [9]:
# Send the POST request to the API endpoint
response = requests.post(api_endpoint, headers=headers, data=json.dumps(payload), timeout=10)

In [10]:
caption = response.json()['caption']
tags = response.json()['tags']

In [11]:
print("Caption: ", caption)
print("Tags: ", tags)

Caption:  a woman putting makeup on her face with lipstick on her lips
Tags:  lip | woman | model | makeup | lipstick | person | face
